In [1]:
print("hello world")

hello world


In [8]:
%pip install kaggle

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re

In [15]:
df = pd.read_csv('/Users/matth/.kaggle/chess_data.zip', usecols=['AN', 'WhiteElo'], compression='zip')
df.sample(5)

,WhiteElo,AN
6184721,1773,1. d4 e6 2. c4 Nf6 3. Nc3 Bb4 4. Qc2 Bxc3+ 5. ...
1344917,1403,1. e3 d6 2. Qf3 Nf6 3. Nc3 g6 4. Bd3 Bg7 5. Nh...
4289761,2037,1. g3 e6 2. Bg2 Nf6 3. e3 d5 4. Ne2 c6 5. O-O ...
1247220,1964,1. d4 g6 2. c4 Bg7 3. e4 c5 4. d5 d6 5. Nc3 Nf...
4420988,2557,1. Nf3 d5 2. d3 Nf6 3. g3 Bg4 4. Bg2 Bxf3 5. B...


In [5]:
letter_2_num = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4, 'f':5, 'g':6, 'h':7}
num_2_letter = {v: k for k, v in letter_2_num.items()}
print(letter_2_num)
print(num_2_letter)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
{0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}


In [7]:
def board_to_rep(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k', 'P', 'R', 'N', 'B', 'Q', 'K']
    layers = []
    for piece in pieces:
        layers.append(create_rep_layer(board, piece))
    board_rep = np.stack(layers, axis=0)
    return board_rep

def create_rep_layer(board, piece):
    s = str(board)
    s = re.sub(f'[^{type}{type.upper()} \n]', '.', s)
    s = re.sub(f'{type}', '-1', s)
    s = re.sub(f'{type.upper()}', '1', s)
    s = re.sub(f'\.', '0', s)

    board_mat = []
    for row in s.split('\n'):
        row = row.split(" ")
        row = [int(x) for x in row]
        board_mat.append(row)
    return np.array(board_mat)

In [8]:
def move_to_rep(move, board):
    board.push_san(move).uci()
    move = str(board.pop())

    from_output_layer = np.zeros((8, 8))
    from_row = 8 - int(move[1])
    from_col = letter_2_num[move[0]]
    from_output_layer[from_row, from_col] = 1

    to_output_layer = np.zeros((8, 8))
    to_row = 8 - int(move[3])
    to_col = letter_2_num[move[2]]
    to_output_layer[to_row, to_col] = 1

    return np.stack([from_output_layer, to_output_layer], axis=0)

In [9]:
def create_move_list(s):
    return re.sub(r'\d*\. ', '', s).split()[:-1]

In [ ]:
chess_data = df[df['WhiteElo'] > 2000]
print(chess_data.shape)
chess_data.sample(10)

NameError: name 'df' is not defined

In [19]:
chess_data = chess_data[['AN']]
chess_data = chess_data[-chess_data['AN'].str.contains('{')]
chess_data = chess_data[chess_data['AN'].str.len() > 20]
print(chess_data.shape[0]) # 800k chess games
chess_data.head(5)

883376


,AN
7,1. d4 d5 2. Nf3 Nf6 3. Bf4 c6 4. e3 Bg4 5. Be2...
8,1. d4 Nf6 2. Bf4 e6 3. e3 d5 4. Nf3 h6 5. Bd3 ...
13,1. b4 e6 2. Bb2 d5 3. Nf3 Nf6 4. b5 Be7 5. e3 ...
26,1. e4 d6 2. d4 Nf6 3. Bd3 g6 4. h4 c5 5. h5 Nx...
39,1. e4 e6 2. d4 d5 3. Nc3 Bb4 4. e5 c5 5. a3 cx...


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [25]:
class ChessDataset(Dataset):
    def __init__(self, games):
        super (ChessDataset, self).__init__()
        self.games = games

    def __len__(self):
        return 40_000
    
    def __getitem__(self, index):
        game_i = np.random.randint(self.game.shpae[0])
        random_game = chess_data['AN'].values[game_i]
        moves = create_move_list(random_game)
        game_state_i = np.random.randint(len(moves)-1)
        next_move = moves[game_state_i]
        moves = moves[:game_state_i]
        board = chess.Board()
        for move in moves:
            board.push_san(move)
        x = board_to_rep(board)
        y = move_to_rep(next_move, board)
        if game_state_i % 2 == 1:
            x *= -1
        return x, y

In [29]:
data_train = ChessDataset(chess_data['AN'])
data_loader = DataLoader(data_train, batch_size=32, shuffle=True)

In [ ]:
class module(nn.Module):

    def __init__(self, hidden_size):
        super(module, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, 3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(hidden_size)
        self.bn2 = nn.BatchNorm2d(hidden_size)
        self.activation1 = nn.SELU()
        self.activation2 = nn.SELU()

    def forward(self,x):
        x_input = torch.clone(x)
        x = self.conv1(x)
        x = self.bn2(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += x_input
        x = self.activation2(x)

        return x